In [1]:
# sample empeddings with pytorch

import torch
import torch.nn as nn
import torch.optim as optim



# lets say you have input [2,3,1]
# 2 can be represent as one hot encoding [0,1,0,0,0,0]
# 3 can be represent as one hot encoding [0,0,1,0,0,0]
# 1 can be represent as one hot encoding [1,0,0,0,0,0]

# so the input can be represent as [[0,1,0,0,0,0], [0,0,1,0,0,0], [1,0,0,0,0,0]]
# vector_size = 6


vector_size = 6 
output_dim = 3

# manual seed
torch.manual_seed(123)

# embeddings
embedding_layer = nn.Embedding(vector_size, output_dim)

# print the embedding layer
print(embedding_layer.weight)


Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [2]:
# get embedding layer for id 3 
print(embedding_layer(torch.tensor([3,0,1])))

tensor([[-0.4015,  0.9666, -1.1481],
        [ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [3]:
with open("the-verdict.txt") as f:
    raw_text = f.read()

print("first 99 characters of raw_text:", raw_text[:99])
# print length of raw_text
print("Length of raw_text:", len(raw_text))

first 99 characters of raw_text: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
Length of raw_text: 20479


In [7]:
import tiktoken
from torch.utils.data import Dataset, DataLoader

class SimpleTokenizerV2:
    def __init__(self):
        self.tokenizer = tiktoken.get_encoding("gpt2")

    def encode(self, text):
        return self.tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    
    def decode(self, tokens):
        return self.tokenizer.decode(tokens)


class InputTargetDataset(Dataset):
    def __init__(self, text, tokenizer, context_size, stride_length):
        self.tokenizer = tokenizer
        self.context_size = context_size
        self.stride_length = stride_length

        self.input_ids = []
        self.target_ids = []

        tokens = self.tokenizer.encode(text)

        for i in range(0, len(tokens) - self.context_size, self.stride_length):
            context = tokens[i:i+self.context_size]
            target = tokens[i+1:i+self.context_size+1]  
            self.input_ids.append(torch.tensor(context))
            self.target_ids.append(torch.tensor(target))


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # i am returning a tensor here!!!
        context = self.input_ids[idx]
        target = self.target_ids[idx]
        return context, target


def create_dataloader(text, batch_size=4, context_size=256, stride_length=128, shuffle=True, drop_last=True, num_workers=0):
    dataset = InputTargetDataset(text, SimpleTokenizerV2(), context_size, stride_length)

    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last, 
        num_workers=num_workers)
    return dataloader

context_size = 4
dataloader = create_dataloader(raw_text, batch_size=8, context_size=context_size, stride_length=context_size, shuffle=False, drop_last=True, num_workers=0)

iterator = iter(dataloader)
input_ids, target_ids = next(iterator)


print("input_ids:\n", input_ids)
# shape
print("input_ids shape:", input_ids.shape)


print("input_ids shape from dataloader:", input_ids.shape)
print("input_ids values:\n", input_ids)

vector_size = 50257
output_dim = 256
token_embedding_layer = nn.Embedding(vector_size, output_dim)
print("token_embedding_layer shape:", token_embedding_layer.weight.shape)

# token embedding of input 
output = token_embedding_layer(input_ids)
print("output shape:", output.shape)



# create positional embedding 
positional_embedding_layer = nn.Embedding(context_size, output_dim)

# get positional embedding 
positional_embedding = positional_embedding_layer(torch.arange(context_size))
print("positional_embedding shape:", positional_embedding.shape)

# with broadcast operation additon is supported
input_embeddings = output + positional_embedding
print("input_embeedings shape:", input_embeddings.shape)





input_ids:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
input_ids shape: torch.Size([8, 4])
input_ids shape from dataloader: torch.Size([8, 4])
input_ids values:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
token_embedding_layer shape: torch.Size([50257, 256])
output shape: torch.Size([8, 4, 256])
positional_embedding shape: torch.Size([4, 256])
input_embeedings shape: torch.Size([8, 4, 256])
